In [ ]:
import root_config as rc

rc.configure()

from detectdd.auth_bigquery import BigQueryClient
from detectdd.serializer import Serializer

try:
    cohort = Serializer().read_cohort()  # need to run 01-cohort.ipynb to produce the cohort
except FileNotFoundError:
    raise Exception("Need to run [01-cohort.ipynb] at least once to create the cohort file in the /out directory")


big_query = BigQueryClient.auth()

cohort.describe()


## kidney diagnoses

In [ ]:
kidney_diagnoses_query = """
SELECT d.subject_id, d.hadm_id, d.icd_code, i.long_title
FROM `physionet-data.mimiciv_hosp.diagnoses_icd` d
JOIN `physionet-data.mimiciv_hosp.d_icd_diagnoses` i ON d.icd_code = i.icd_code
WHERE i.long_title LIKE '%kidney%'
"""

kidney_diagnoses_results = big_query.query(kidney_diagnoses_query).result().to_dataframe()


In [ ]:
kidney_diagnoses_results

## kidney prescriptions

In [ ]:
kidney_prescriptions_query = """
WITH kidney_patients AS (
    SELECT DISTINCT d.subject_id, d.hadm_id
    FROM `physionet-data.mimiciv_hosp.diagnoses_icd` d
    JOIN `physionet-data.mimiciv_hosp.d_icd_diagnoses` i ON d.icd_code = i.icd_code
    WHERE i.long_title LIKE '%kidney%'
    LIMIT 100000
)

SELECT p.subject_id, p.hadm_id, p.drug
FROM `physionet-data.mimiciv_hosp.prescriptions` p
JOIN kidney_patients kp ON p.subject_id = kp.subject_id AND p.hadm_id = kp.hadm_id
LIMIT 100000
"""

kidney_prescriptions_results = big_query.query(kidney_prescriptions_query).result().to_dataframe()


In [ ]:
kidney_prescriptions_results

## kidney lab

In [ ]:
kidney_lab_query = """
WITH kidney_patients AS (
    SELECT DISTINCT d.subject_id, d.hadm_id
    FROM `physionet-data.mimiciv_hosp.diagnoses_icd` d
    JOIN `physionet-data.mimiciv_hosp.d_icd_diagnoses` i ON d.icd_code = i.icd_code
    WHERE i.long_title LIKE '%kidney%'
    LIMIT 100000
)

SELECT l.subject_id, l.hadm_id, i.label, l.value, l.valuenum, l.valueuom
FROM `physionet-data.mimiciv_hosp.labevents` l
JOIN `physionet-data.mimiciv_hosp.d_labitems` i ON l.itemid = i.itemid
WHERE i.label IN ('Creatinine', 'BUN') 
AND l.subject_id IN (SELECT subject_id FROM kidney_patients)
AND l.hadm_id IN (SELECT hadm_id FROM kidney_patients)
LIMIT 100000
"""

kidney_lab_results = big_query.query(kidney_lab_query).result().to_dataframe()


In [ ]:
kidney_lab_results

In [ ]:
merged_data = kidney_diagnoses_results.merge(kidney_lab_results, on=['subject_id', 'hadm_id'], how='left').merge(kidney_prescriptions_results, on=['subject_id', 'hadm_id'], how='left')

In [ ]:
merged_data

In [ ]:
# merged_data.to_csv('merged_kidney_data.csv', index=False)

In [ ]:
drug_combinations_query = """
WITH kidney_patients AS (
    SELECT DISTINCT d.subject_id, d.hadm_id
    FROM `physionet-data.mimiciv_hosp.diagnoses_icd` d
    JOIN `physionet-data.mimiciv_hosp.d_icd_diagnoses` i ON d.icd_code = i.icd_code
    WHERE i.long_title LIKE '%kidney%'
),

drugs_within_same_hour AS (
    SELECT 
        p1.subject_id,
        p1.hadm_id,
        p1.drug AS drug_a,
        p2.drug AS drug_b,
        p1.starttime AS starttime_a,
        p2.starttime AS starttime_b,
        COUNT(*) OVER (PARTITION BY p1.subject_id, p1.hadm_id, p1.drug, p2.drug) AS event_count
    FROM `physionet-data.mimiciv_hosp.prescriptions` p1
    JOIN `physionet-data.mimiciv_hosp.prescriptions` p2 
        ON p1.subject_id = p2.subject_id AND p1.hadm_id = p2.hadm_id
    WHERE p1.drug <> p2.drug 
        AND ABS(TIMESTAMP_DIFF(p1.starttime, p2.starttime, MINUTE)) <= 60
        AND p1.subject_id IN (SELECT subject_id FROM kidney_patients)
        AND p1.hadm_id IN (SELECT hadm_id FROM kidney_patients)
)

SELECT * FROM drugs_within_same_hour
LIMIT 100000
"""

drugs_combinations_results = big_query.query(drug_combinations_query).result().to_dataframe()

drugs_combinations_results.describe()


In [ ]:
drugs_combinations_results